Given that the mechanism traverses the complete mesh, we now make it sensible to the order of $T$.



In [16]:
import openmesh as om
import numpy as np
import matplotlib.pyplot as plt

mesh = om.read_trimesh('T4.off')
V=mesh.points()
X=V[:,0]
Y=V[:,1]

Initialize 
* active nodes
* active face list
* $T$ & $S$ values: (1) at nodes, and (2) for faces


In [17]:
def initActiveNodes():
    ActiveNodeList = []
    for vh in mesh.vertices():
        k = vh.idx()
        if V[k,0] == 0 or V[k,1] == 1:
            ActiveNodeList.append(vh)
    return ActiveNodeList

def initActiveFaceList():
    ActiveFaceList = []

    i=0
    for fh in mesh.faces():
        counter = 0
    
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                # print(vh.idx(),' in ', fh.idx())                
                counter = counter + 1
            
        if counter == 2:
            ActiveFaceList.append(fh)

    return ActiveFaceList

def init_TS_nodes():
    T = np.zeros(len(V))+7
    S = np.zeros(len(V))+7

    for vh in ActiveNodeList:    
        k = vh.idx()
        if V[k,0] == 0:
            T[k] = 0
            S[k] = 0
        elif V[k,1] == 1:
            T[k] = V[k,0]
            S[k] = 0
    
    return T, S

def init_TS_faces():
    Tf = np.zeros(len(mesh.faces()))+7
    Sf = np.zeros(len(mesh.faces()))+7

    for fh in ActiveFaceList:
        T_ABC = 0
        S_ABC = 0
        counter = 0
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                T_ABC = T_ABC + T[vh.idx()]
                S_ABC = S_ABC + S[vh.idx()]            
                counter = counter + 1
            
        if counter != 2:
            print('ERR at face', fh.idx())
        else:
            Tf[fh.idx()] = T_ABC / 2
            Sf[fh.idx()] = S_ABC / 2
            
    return Tf, Sf
    
# routine to get active face indices    
def getActiveFaceIndices():
    I_list = []
    for fh in ActiveFaceList:
        I_list.append(fh.idx())
        
    return I_list

    
def getActiveNodeIndices():
    I_list = []
    for fh in ActiveNodeList:
        I_list.append(fh.idx())

    return I_list


def getProcessedFaceIndices():
    I_list = []
    for fh in ProcessedFaceList:
        I_list.append(fh.idx())
        
    return I_list

def getObsoleteFaceIndices():
    I_list = []
    for fh in ObsoleteFaceList:
        I_list.append(fh.idx())
        
    return I_list

def get_T_C_dummy(T_A, T_B):
    return (T_A + T_B)/2 + 10

# ... update faces: don inline    



ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()
    
T, S = init_TS_nodes()
Tf, Sf = init_TS_faces()

I_active_faces = getActiveFaceIndices()
print('active faces:', I_active_faces)


active faces: [1, 4, 5, 8, 11, 22, 40, 74, 75]


For overseeing the algorithm one needs to track the active faces. 

Generalize the first step in the overall while cycle.



If a new face is appended, then the corresponding approximate solution variables are actualized.

Late face: a face that comes at top of the list, but whose node are already calculated, as the nodes are parts of neighbor faces that have been handled earlier. Such a face becomes obsolete, so the algorithm passes to the next face. Just for tracking, an obsolete face list is generated.

In [36]:
ProcessedFaceList = []
ObsoleteFaceList = []

print('len: list(I[K]) \t \t (id_A, id_B) -> id_C \n \n \n')
j=0
while len(ActiveFaceList)>0:
    j=j+1
    
    # (0) update the index
    I_active_faces = getActiveFaceIndices()

    I = np.array(I_active_faces)
    K = np.argsort(Tf[I], axis=0)

    # print('(len)', len(I_active_faces),': (f)', list(I[K]))
    
    # (1) obtain id of next face
    next_face_id = (I[K])[0]
    fh = mesh.face_handle(next_face_id)
    
    # quick check whether the face is obsolete,
    # which is a normal situation that should not be handled as an error
    # but by moving on to the next face

    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            counter = counter + 1    
    if counter == 3:
        print('remove obsolete face:', fh.idx())
        ObsoleteFaceList.append(fh)
        ActiveFaceList.remove(fh)
        continue
            
    # (2) calculate value of outgoing vertex
    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            # print(vh.idx(),'ingoing')
            if counter == 0:
                counter = counter + 1
                id_A = vh.idx()
                T_A = T[vh.idx()]                
            elif counter == 1:
                counter = counter + 1
                id_B = vh.idx()
                T_B = T[vh.idx()]
            else:
                print('ERR: too many ingoing nodes')
                
        else:
            # print(vh.idx(),'outgoing')        
            id_C = vh.idx()
            vh_C = vh
            
    if counter == 2:
        T_C = get_T_C_dummy(T_A, T_B)
        T[id_C] = T_C
    else:
        print('ERR: missing inputs for calculating T_C')
        # T[id_C] = j
        
    # T[id_C] = j
    
    # (3) handle the list of active faces
    if (not vh_C in ActiveNodeList):
        ActiveNodeList.append(vh_C) # also ! 
    else:
        print('node already listed:', vh_C.idx())
    
    ActiveFaceList.remove(fh)
    ProcessedFaceList.append(fh)

    # (3b) agregate new faces to the list
    f_list = []
    print(len(I_active_faces),': ', list(I[K]),'; \t (', id_A, ',', id_B, ') ->', id_C)    
    for fvh in mesh.vf(vh_C):
        f_list.append(fvh.idx())
        # print('fv', fvh.idx())
        if (not fvh in ActiveFaceList) and (not fvh in ProcessedFaceList):
            counter = 0
            for vh in mesh.fv(fvh):
                if vh in ActiveNodeList:
                    counter = counter + 1
        
            if counter == 2:
                ActiveFaceList.append(fvh)
                print('new face:', fvh.idx())
                
                for vh in mesh.fv(fvh):
                    if vh in ActiveNodeList:
                        if counter == 0:
                            T_A = T[vh.idx()]
                            counter = counter + 1
                        elif counter == 1:
                            T_B = T[vh.idx()]
                    Tf[fvh.idx()] = (T_A + T_B)/2 + 3              
            
            elif counter == 3:
                print('face duplicate registered:', fvh.idx())                
        
    print('\t \t f_list:', f_list, '\n\n\n')


        
    if j == 7:
        I_active_faces = getActiveFaceIndices()
        I = np.array(I_active_faces)
        K = np.argsort(Tf[I], axis=0)

        Z = np.vstack([I[K],Tf[I[K]]]) # ,Sf[I[K]]])
        print(Z.transpose())        


len: list(I[K]) 	 	 (id_A, id_B) -> id_C 
 
 



The outgoing vertex now becomes a potential ingoing vertex. Therefore, all connected faces to that resolved outgoing node are checked, whether (1) they are available (not yet listed in the active or processed list), (2) there are two ingoing nodes (the newly agregated ingoing node and another one.).


array([ 3.       ,  0.5      , 55.3046875, 38.       ,  3.5      ,
        0.       ,  3.       ,  3.       ,  0.       ,  3.       ,
       20.5      ,  0.       , 37.28125  , 31.75     , 31.75     ,
       23.       , 24.25     , 49.484375 , 41.125    , 49.484375 ,
       44.34375  , 25.1875   ,  2.5      , 31.40625  , 24.1875   ,
       37.28125  , 38.       , 33.       , 41.6875   , 20.5      ,
       13.       , 13.       ,  8.       ,  3.       , 13.       ,
        3.       , 23.       , 23.       , 23.       , 23.125    ,
        1.5      ,  8.5      ,  4.5      ,  3.       ,  3.       ,
       31.40625  , 38.296875 ,  6.5      , 11.75     , 18.625    ,
       17.625    ,  9.75     ,  5.5      , 16.5      ,  9.75     ,
        4.5      , 10.75     ,  5.5      , 17.625    , 10.75     ,
        6.5      , 18.625    , 24.1875   , 30.375    , 41.6875   ,
       33.       , 23.125    , 30.375    , 13.       , 16.5      ,
       13.       , 13.       ,  8.5      ,  3.       ,  0.    

## Checksum

In [34]:

I_active_nodes = getActiveNodeIndices()
print('active nodes (usorted):', I_active_nodes)
print(len(I_active_nodes),': ',np.sort(I_active_nodes))

I_faces = getProcessedFaceIndices()
I_faces_obsolete = getObsoleteFaceIndices()
print('check completeness:', len(I_faces), len(I_faces_obsolete))
print('faces, processed:', np.sort(I_faces))
print('faces, obsolete:', np.sort(I_faces_obsolete))




active nodes (usorted): [0, 5, 10, 15, 20, 25, 26, 27, 28, 29, 30, 34, 38, 46, 42, 47, 48, 1, 6, 11, 21, 16, 49, 22, 23, 24, 43, 44, 45, 31, 35, 39, 17, 18, 19, 2, 7, 12, 40, 41, 36, 13, 14, 32, 37, 8, 3, 33, 9, 4]
50 :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
check completeness: 40 40
faces, processed: [ 0  3  4  5  6  8 10 11 12 13 15 18 22 24 25 30 31 33 35 37 40 41 45 47
 49 51 53 55 57 58 59 63 66 67 70 73 74 75 77 79]
faces, obsolete: [ 1  2  7  9 14 16 17 19 20 21 23 26 27 28 29 32 34 36 38 39 42 43 44 46
 48 50 52 54 56 60 61 62 64 65 68 69 71 72 76 78]
